# Non-lexical vocables

In [2]:
#!pip install pyspark
#!pip install wordcloud
import numpy as np
import pandas as pd
from pyspark.sql import HiveContext
from pyspark import SparkContext
#hive_context = HiveContext(sc)
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
sys.path.append(os.getcwd())

import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
pwd

'/home/jupyter/w266_project_topmusichits/notebooks'

In [5]:
df = pd.read_csv("/home/jupyter/w266_project_topmusichits/data/interim/songs_charts_new_2000s.csv")
df2 = pd.read_csv("/home/jupyter/w266_project_topmusichits/data/interim/songs_charts_new_2000s.csv")


In [6]:
df.columns

Index(['song_name', 'artist_name', 'lyrics', 'album_name', 'genre', 'subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'song_year', 'trending_year', 'weeks_on_chart', 'hit',
       'star'],
      dtype='object')

In [7]:
df['lyrics_lower'] = df['lyrics'].str.lower()

In [8]:
#Non-lexical vocables identified in the data:
#     'nah'
#     'na'
#     'oh'
#     'ooh'
#     'oo'
#     'uh'
#     'la'
#     'ah'
#     'doo'
#     'da'

df['nah']=df.lyrics_lower.str.count("nah")
df['na']=df.lyrics_lower.str.count("na")
df['oh']=df.lyrics_lower.str.count("oh")
df['ooh']=df.lyrics_lower.str.count("ooh")
df['oo']=df.lyrics_lower.str.count("oo")
df['uh']=df.lyrics_lower.str.count("uh")
df['la']=df.lyrics_lower.str.count("la")
df['ah']=df.lyrics_lower.str.count("ah")
df['doo']=df.lyrics_lower.str.count("doo")
df['da']=df.lyrics_lower.str.count("da")
df['num_nonlexvoc_1'] = df['nah'] + df['na'] + df['oh'] + df['ooh'] + df['oo'] + df['uh'] + df['la'] + df['ah'] + df['doo'] + df['da']
df['nah nah nah']=df.lyrics_lower.str.count("nah nah nah")
df['na na na']=df.lyrics_lower.str.count("na na na")
df['oh oh oh']=df.lyrics_lower.str.count("oh oh oh")
df['ooh ooh ooh']=df.lyrics_lower.str.count("ooh ooh ooh")
df['oo oo oo']=df.lyrics_lower.str.count("oo oo oo")
df['uh uh uh']=df.lyrics_lower.str.count("uh uh uh")
df['la la la']=df.lyrics_lower.str.count("la la la")
df['ah ah ah']=df.lyrics_lower.str.count("ah ah ah")
df['doo doo doo']=df.lyrics_lower.str.count("doo doo doo")
df['da da da']=df.lyrics_lower.str.count("da da da")
df['num_nonlexvoc_3'] = df['nah nah nah'] + df['na na na'] + df['oh oh oh'] + df['ooh ooh ooh'] + df['oo oo oo'] + df['uh uh uh'] + df['la la la'] + df['ah ah ah'] + df['doo doo doo'] + df['da da da']

In [9]:
df.head()

,song_name,artist_name,lyrics,album_name,genre,subgenre,danceability,energy,key,loudness,...,na na na,oh oh oh,ooh ooh ooh,oo oo oo,uh uh uh,la la la,ah ah ah,doo doo doo,da da da,num_nonlexvoc_3
0,?,MF DOOM,NA My servants began to forge what was to beco...,Operation: Doomsday (Complete),rap,gangster rap,0.532,0.629,10,-7.955,...,0,0,0,0,0,0,0,0,0,0
1,0 To 100 / The Catch Up,Drake,NA NA Fuck bein' on some chill shit We go 0 to...,0 To 100 / The Catch Up,r&b,urban contemporary,0.559,0.716,11,-3.964,...,0,0,0,0,0,0,0,0,0,0
2,10%,KAYTRANADA,You keep on takin' from me But where's my ten ...,BUBBA,r&b,urban contemporary,0.794,0.757,6,-6.644,...,0,0,0,0,0,0,0,0,0,0
3,1000 Nights,FRENSHIP,When the fever's gone And the road's to dust I...,1000 Nights,pop,indie poptimism,0.594,0.709,6,-9.541,...,0,0,8,0,0,0,0,0,0,8
4,1000 Rounds,Pouya,"Bullets fly, legs buckle, bodies pile to the s...",1000 Rounds,rap,gangster rap,0.825,0.761,8,-5.389,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.shape

(8586, 46)

In [11]:
print(df['num_nonlexvoc_1'].sum())

print(df['num_nonlexvoc_3'].sum())

282937
1986


In [12]:
pd.crosstab(df.hit, df.num_nonlexvoc_1)

num_nonlexvoc_1,0,1,2,3,4,5,6,7,8,9,...,593,616,618,732,772,864,924,960,1256,1280
hit,,,,,,,,,,,,,,,,,,,,,
0,171,135,169,223,256,223,250,246,247,235,...,1,1,0,1,0,1,0,0,0,0
1,7,5,18,18,21,27,26,32,29,30,...,0,0,1,0,1,0,1,1,1,1


In [13]:
df.loc[df['num_nonlexvoc_1'] >=1, 'num_nonlexvoc_1b'] = 1
df.loc[df['num_nonlexvoc_1'] ==0, 'num_nonlexvoc_1b'] = 0

In [14]:
pd.crosstab(df.hit, df.num_nonlexvoc_1b)

num_nonlexvoc_1b,0.0,1.0
hit,,
0,171,6807
1,7,1601


In [15]:
pd.crosstab(df.hit, df.num_nonlexvoc_1b, normalize='index')

num_nonlexvoc_1b,0.0,1.0
hit,,
0,0.024506,0.975494
1,0.004353,0.995647


In [16]:
df.loc[df['nah'] >=1, 'nah_b'] = 1
df.loc[df['nah'] ==0, 'nah_b'] = 0

pd.crosstab(df.hit, df.nah_b, normalize='index')

nah_b,0.0,1.0
hit,,
0,0.961164,0.038836
1,0.937189,0.062811


In [17]:
df.loc[df['na'] >=1, 'na_b'] = 1
df.loc[df['na'] ==0, 'na_b'] = 0

pd.crosstab(df.hit, df.na_b, normalize='index')

na_b,0.0,1.0
hit,,
0,0.346948,0.653052
1,0.212687,0.787313


In [18]:
bins = [-np.inf, 0, 40, 80, np.inf]
labels=['0', '1-40','41-80', '80+']
df['num_nonlexvoc_1_ranges'] = pd.cut(df['num_nonlexvoc_1'], bins=bins, labels=labels)
print(df['num_nonlexvoc_1_ranges'].value_counts())

1-40     6477
41-80    1220
80+       711
0         178
Name: num_nonlexvoc_1_ranges, dtype: int64


In [19]:
pd.crosstab(df.hit, df['num_nonlexvoc_1_ranges'], normalize='index')

num_nonlexvoc_1_ranges,0,1-40,41-80,80+
hit,,,,
0,0.024506,0.797220,0.121955,0.056320
1,0.004353,0.568408,0.229478,0.197761


In [20]:
bins = [-np.inf, 0, 40, 80, np.inf]
labels=['Zero non-lexical vocables', 'Low non-lexical vocables','Medium non-lexical vocables', 'High non-lexical vocables']
df['num_nonlexvoc_1_r_label'] = pd.cut(df['num_nonlexvoc_1'], bins=bins, labels=labels)
print(df['num_nonlexvoc_1_r_label'].value_counts())

Low non-lexical vocables       6477
Medium non-lexical vocables    1220
High non-lexical vocables       711
Zero non-lexical vocables       178
Name: num_nonlexvoc_1_r_label, dtype: int64


In [25]:
df2['lyrics_lower'] = df2['lyrics'].str.lower()
df2['nah']=df2.lyrics_lower.str.count("nah")
df2['na']=df2.lyrics_lower.str.count("na")
df2['oh']=df2.lyrics_lower.str.count("oh")
df2['ooh']=df2.lyrics_lower.str.count("ooh")
df2['oo']=df2.lyrics_lower.str.count("oo")
df2['uh']=df2.lyrics_lower.str.count("uh")
df2['la']=df2.lyrics_lower.str.count("la")
df2['ah']=df2.lyrics_lower.str.count("ah")
df2['doo']=df2.lyrics_lower.str.count("doo")
df2['da']=df2.lyrics_lower.str.count("da")
df2['num_nonlexvoc_1'] = df2['nah'] + df2['na'] + df2['oh'] + df2['ooh'] + df2['oo'] + df2['uh'] + df2['la'] + df2['ah'] + df2['doo'] + df2['da']
df2 = df2.drop(columns=['lyrics_lower', 'nah', 'na', 'oh', 'ooh', 'oo', 'uh' , 'la', 'ah', 'doo', 'da'])
bins = [-np.inf, 0, 40, 80, np.inf]
labels=['Zero non-lexical vocables', 'Low non-lexical vocables','Medium non-lexical vocables', 'High non-lexical vocables']
df2['num_nonlexvoc_1_r_label'] = pd.cut(df2['num_nonlexvoc_1'], bins=bins, labels=labels)


In [26]:
df2.columns

Index(['song_name', 'artist_name', 'lyrics', 'album_name', 'genre', 'subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'song_year', 'trending_year', 'weeks_on_chart', 'hit',
       'star', 'num_nonlexvoc_1', 'num_nonlexvoc_1_r_label'],
      dtype='object')

In [30]:
df2.to_csv('/home/jupyter/w266_project_topmusichits/data/interim/songs_charts_new_2000s.csv')

# Trigrams

In [17]:
pd.crosstab(df.hit, df.num_nonlexvoc_3)

num_nonlexvoc_3,0,1,2,3,4,5,6,7,8,9,...,27,28,30,34,35,38,48,80,88,99
hit,,,,,,,,,,,,,,,,,,,,,
0,6785,50,24,15,20,8,15,2,9,4,...,0,1,3,1,2,1,1,0,1,0
1,1551,9,12,3,10,0,2,1,5,0,...,1,0,1,0,0,0,0,1,0,1


In [18]:
df.loc[df['num_nonlexvoc_3'] >=1, 'num_nonlexvoc_3b'] = 1
df.loc[df['num_nonlexvoc_3'] ==0, 'num_nonlexvoc_3b'] = 0

In [19]:
pd.crosstab(df.hit, df.num_nonlexvoc_3b, normalize='index')

num_nonlexvoc_3b,0.0,1.0
hit,,
0,0.972342,0.027658
1,0.964552,0.035448


In [20]:
df.loc[df['nah nah nah'] >=1, 'nah nah nah_b'] = 1
df.loc[df['nah nah nah'] ==0, 'nah nah nah_b'] = 0

pd.crosstab(df.hit, df['nah nah nah_b'], normalize='index')

nah nah nah_b,0.0,1.0
hit,,
0,0.99957,0.00043
1,1.00000,0.00000


In [27]:
bins = [-np.inf, 0, 1, 3, 7, np.inf]
labels=['0', '1', '2-3', '4-7','8+']
df['num_nonlexvoc_3_ranges'] = pd.cut(df['num_nonlexvoc_3'], bins=bins, labels=labels)
print(df['num_nonlexvoc_3_ranges'].value_counts())

0      8336
8+       79
1        59
4-7      58
2-3      54
Name: num_nonlexvoc_3_ranges, dtype: int64


In [28]:
pd.crosstab(df.hit, df['num_nonlexvoc_3_ranges'], normalize='index')

num_nonlexvoc_3_ranges,0,1,2-3,4-7,8+
hit,,,,,
0,0.972342,0.007165,0.005589,0.006449,0.008455
1,0.964552,0.005597,0.009328,0.008085,0.012438


In [29]:
pd.crosstab(df.hit, df['num_nonlexvoc_3_ranges'])

num_nonlexvoc_3_ranges,0,1,2-3,4-7,8+
hit,,,,,
0,6785,50,39,45,59
1,1551,9,15,13,20


In [20]:
df.to_csv('df.csv')